# 1.1 핵심 개념
* 로지스틱 회귀분석 : 종속변수가 범주형 자료일 경우 적용하는 회귀모델
* 원자료를 확률→오즈(p/(1-p))→로그 로 변환하여 선형회귀모델을 적용한 모델
* 해석은 다시 지수로 변환하여 원데이터로 전환한 모델

# 1.3 분석 코드

In [1]:
# 분석데이터 준비
mport warnings
warnings.filterwarnings('ignore')
import pandas as pd
data=pd.read_csv("../data/breast-cancer-wisconsin.csv", encoding='utf-8')
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
X=data[data.columns[1:-1]]
y=data[['Class']]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y, random_state=42)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler_minmax=MinMaxScaler()
scaler_minmax.fit(X_train)
X_scaled_minmax_train=scaler_minmax.transform(X_train)
X_scaled_minmax_test=scaler_minmax.transform(X_test)

In [6]:
#기본모델 적용
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_scaled_minmax_train, y_train)
pred_train=model.predict(X_scaled_minmax_train)
model.score(X_scaled_minmax_train, y_train)

0.97265625

In [8]:
pred_test=model.predict(X_scaled_minmax_test)
model.score(X_scaled_minmax_test, y_test)

0.9590643274853801

In [9]:
from sklearn.metrics import confusion_matrix
confusion_train=confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n",confusion_train)

훈련데이터 오차행렬 : 
 [[328   5]
 [  9 170]]


In [11]:
from sklearn.metrics import classification_report
cfreport_train=classification_report(y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       333
           1       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [12]:
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  2  58]]


In [13]:
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [15]:
#Grid Search
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(LogisticRegression(),
                        {'C':[0.001,0.01,0.1,1,10,100]},
                        cv=5,
                        return_train_score=True)
grid_search.fit(X_scaled_minmax_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             return_train_score=True)

In [22]:
print(f"Best Parameter : {grid_search.best_params_}")
print(f"Best Score : {grid_search.best_score_:.4f}")
print(f"TestSet Score : {grid_search.score(X_scaled_minmax_test, y_test):.4f}")

Best Parameter : {'C': 10}
Best Score : 0.9726
TestSet Score : 0.9591


In [26]:
#Random Serch
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(LogisticRegression(),
                                param_distributions={'C':randint(low=0.001, high=100)},
                                n_iter=100, cv=5,
                                return_train_score=True)
random_search.fit(X_scaled_minmax_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EBA2FC4B38>},
                   return_train_score=True)

In [27]:
print(f"Best Parameter : {random_search.best_params_}")
print(f"Best Score : {random_search.best_score_}")
print(f"TestSet Score : {random_search.score(X_scaled_minmax_test, y_test)}")

Best Parameter : {'C': 12}
Best Score : 0.9745478774033887
TestSet Score : 0.9590643274853801
